
# Multi‑Agent Trash Collection Simulator (2D Grid)

This notebook provides an **extensible, agent‑based simulation** of trash collection in a small city grid.  
It includes **trucks**, **bins**, **traffic lights**, and **dump sites**, plus:
- A discrete‑time simulator (step-based)
- A* pathfinding on a road grid
- Traffic lights with configurable phases (NS vs EW)
- Bins that **fill over time** and can be **picked up** by trucks
- Trucks with limited capacity that **plan routes** (greedy nearest by default) and **unload** at dumps
- Matplotlib visualization (static or animated)
- Optional interactive widgets (if `ipywidgets` is available)
- A simple **Policy API** you can extend to implement smarter multi‑agent coordination

> You can start by running the **Scenario Setup** cell and then the **Run / Visualize** cells.


In [ ]:

import math
import random
import heapq
from dataclasses import dataclass, field
from typing import Dict, Tuple, List, Optional, Set
from collections import deque

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

try:
    from IPython.display import display, clear_output
    from ipywidgets import interact, IntSlider, Button, HBox, VBox, Checkbox, Play, jslink, Dropdown
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False

# Reproducibility
random.seed(42)
np.random.seed(42)

# Small utility
def manhattan(a: Tuple[int,int], b: Tuple[int,int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])



## Pathfinding (A* on Grid Roads)
We compute shortest paths **on the road graph** (4-neighborhood). Traffic lights are enforced during **movement**, not during path planning, to keep A* static.


In [ ]:

def astar_road_path(road: np.ndarray, start: Tuple[int,int], goal: Tuple[int,int]) -> Optional[List[Tuple[int,int]]]:
    """
    Compute a path from start to goal using A* over road cells.
    Returns a list of grid coords including start and goal; or None.
    """
    h, w = road.shape
    if not (0 <= start[0] < h and 0 <= start[1] < w): return None
    if not (0 <= goal[0] < h and 0 <= goal[1] < w): return None
    if not road[start] or not road[goal]: return None

    def neighbors(r, c):
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < h and 0 <= nc < w and road[nr, nc]:
                yield (nr, nc)

    open_heap = []
    g = {start: 0}
    f = {start: manhattan(start, goal)}
    came = {}
    heapq.heappush(open_heap, (f[start], start))

    while open_heap:
        _, cur = heapq.heappop(open_heap)
        if cur == goal:
            # reconstruct
            path = [cur]
            while cur in came:
                cur = came[cur]
                path.append(cur)
            path.reverse()
            return path

        for nb in neighbors(*cur):
            ng = g[cur] + 1  # cost per move
            if ng < g.get(nb, 1e18):
                g[nb] = ng
                f[nb] = ng + manhattan(nb, goal)
                came[nb] = cur
                heapq.heappush(open_heap, (f[nb], nb))
    return None



## Entities
- **TrafficLight** toggles between NS/EW green phases.
- **Bin** accumulates trash over time and can be serviced.
- **Truck** moves along a planned route, collects, and unloads at dumps.


In [ ]:

@dataclass
class TrafficLight:
    pos: Tuple[int,int]
    ns_green_steps: int = 12
    ew_green_steps: int = 12
    amber_steps: int = 2          # For illustration (no directional pass during amber)
    phase: str = "NS"             # 'NS' or 'EW' or 'AMBER'
    timer: int = 0                # time elapsed in current phase

    def step(self):
        self.timer += 1
        if self.phase == "NS":
            if self.timer >= self.ns_green_steps:
                self.phase = "AMBER"
                self.timer = 0
        elif self.phase == "EW":
            if self.timer >= self.ew_green_steps:
                self.phase = "AMBER"
                self.timer = 0
        else:  # AMBER
            if self.timer >= self.amber_steps:
                # switch to opposite green
                self.phase = "EW" if self.phase == "AMBER" else "NS"
                # But we need to know which was previous. Use a trick:
                # If timer switched from AMBER after NS, go EW; after EW, go NS.
                # We'll store last_g in a small state machine. Simpler: alternate.
                # To alternate properly, track previous:
                pass  # We'll handle alternation via CitySim maintaining last phase.
                # Note: We'll override this behavior in CitySim.step_traffic_lights()


@dataclass
class Bin:
    id: int
    pos: Tuple[int,int]
    capacity: float = 100.0
    level: float = 0.0
    fill_rate: float = 0.5   # per step

    def fill(self):
        self.level = min(self.capacity, self.level + self.fill_rate)

    def is_over_threshold(self, threshold: float) -> bool:
        return (self.level / self.capacity) >= threshold

    def empty(self) -> float:
        amount = self.level
        self.level = 0.0
        return amount


@dataclass
class Truck:
    id: int
    pos: Tuple[int,int]
    capacity: float = 500.0
    load: float = 0.0
    speed_cells_per_step: int = 1
    state: str = "idle"   # idle, to_bin, collecting, to_dump, unloading
    route: List[Tuple[int,int]] = field(default_factory=list)
    target_bin_id: Optional[int] = None

    distance_travelled: int = 0
    waits_at_red: int = 0
    serviced_bins: int = 0

    def has_capacity_for(self, amount: float) -> bool:
        return (self.load + amount) <= self.capacity

    def at_destination(self) -> bool:
        return len(self.route) <= 1  # current cell only

    def step_move(self, road, traffic_light_query):
        """Move along route up to speed if the next step is allowed (green or no light)."""
        steps = self.speed_cells_per_step
        while steps > 0 and len(self.route) > 1:
            cur = self.route[0]
            nxt = self.route[1]
            # Check traffic light permission
            if not movement_allowed(cur, nxt, traffic_light_query):
                self.waits_at_red += 1
                break
            # advance
            self.route.pop(0)
            self.pos = nxt
            self.distance_travelled += 1
            steps -= 1

def movement_allowed(a: Tuple[int,int], b: Tuple[int,int], traffic_light_query):
    """Return True if movement from a->b is allowed based on the current light at the intersection (if any).
    Rule: If moving through a light cell, respect its phase. If no light, always allowed.
    If moving horizontally (same row): allowed on 'EW' green; vertically: 'NS' green.
    Amber = stop.
    """
    light = traffic_light_query(b)
    if light is None:
        return True
    dr = b[0] - a[0]
    dc = b[1] - a[1]
    if (dr == 0 and dc == 0):
        return True
    horizontal = (dr == 0 and abs(dc) == 1)
    vertical   = (dc == 0 and abs(dr) == 1)
    if light.phase == "AMBER":
        return False
    if horizontal and light.phase == "EW":
        return True
    if vertical and light.phase == "NS":
        return True
    return False



## CitySim Core
The main simulator maintains the grid, agents, and step logic:
1. Update traffic lights  
2. Fill bins  
3. Let trucks plan/act/move  
4. Collect stats  


In [ ]:

class CitySim:
    def __init__(self, height=40, width=60):
        self.h = height
        self.w = width
        # Roads: True = drivable
        self.road = np.zeros((self.h, self.w), dtype=bool)
        self.lights: Dict[Tuple[int,int], TrafficLight] = {}
        self.bins: Dict[int, Bin] = {}
        self.dumps: List[Tuple[int,int]] = []
        self.trucks: Dict[int, Truck] = {}
        self.t = 0
        self.stats = {
            "collected_total": 0.0,
            "truck_waits": 0,
        }
        # Internal bookkeeping for traffic light alternation
        self._light_last_green: Dict[Tuple[int,int], str] = {}

    # --- Construction helpers ---
    def set_roads_mesh(self, spacing=6):
        self.road[:, ::spacing] = True
        self.road[::spacing, :] = True

    def add_light(self, r, c, ns_green=12, ew_green=12, amber=2, initial="NS"):
        tl = TrafficLight((r,c), ns_green, ew_green, amber, initial, 0)
        self.lights[(r,c)] = tl
        self._light_last_green[(r,c)] = initial

    def add_bin(self, pos, capacity=100.0, level=0.0, fill_rate=0.5):
        bid = 0 if not self.bins else (max(self.bins.keys()) + 1)
        self.bins[bid] = Bin(bid, pos, capacity, level, fill_rate)
        return bid

    def add_dump(self, pos):
        self.dumps.append(pos)

    def add_truck(self, pos, capacity=500.0, speed=1):
        tid = 0 if not self.trucks else (max(self.trucks.keys()) + 1)
        self.trucks[tid] = Truck(tid, pos, capacity, 0.0, speed)
        return tid

    def is_road(self, rc):
        r, c = rc
        return (0 <= r < self.h and 0 <= c < self.w and self.road[r, c])

    def traffic_light_at(self, rc) -> Optional[TrafficLight]:
        return self.lights.get(rc, None)

    # --- Simulation Steps ---
    def step_traffic_lights(self):
        # Implement proper NS <-> AMBER <-> EW alternation
        for pos, tl in self.lights.items():
            tl.timer += 1
            if tl.phase == "NS" and tl.timer >= tl.ns_green_steps:
                tl.phase = "AMBER"
                tl.timer = 0
                self._light_last_green[pos] = "NS"
            elif tl.phase == "EW" and tl.timer >= tl.ew_green_steps:
                tl.phase = "AMBER"
                tl.timer = 0
                self._light_last_green[pos] = "EW"
            elif tl.phase == "AMBER" and tl.timer >= tl.amber_steps:
                prev = self._light_last_green[pos]
                tl.phase = "EW" if prev == "NS" else "NS"
                tl.timer = 0

    def step_bins(self):
        for b in self.bins.values():
            b.fill()

    # --- Policy Hook (assignments / decisions) ---
    def plan_truck(self, truck: Truck, threshold=0.6):
        # If load near full, go dump
        if truck.load >= truck.capacity * 0.9:
            self.route_to_nearest_dump(truck)
            truck.state = "to_dump"
            return

        # If already en-route, keep going
        if truck.route and not truck.at_destination():
            return

        # If at dump and has load, unload
        if truck.pos in self.dumps and truck.load > 0 and truck.at_destination():
            truck.state = "unloading"
            truck.load = 0.0
            # After unloading, clear any target and go idle
            truck.target_bin_id = None
            truck.state = "idle"
            return

        # If at a bin target and arrived, collect
        if truck.target_bin_id is not None and truck.at_destination():
            b = self.bins.get(truck.target_bin_id, None)
            if b and b.pos == truck.pos:
                truck.state = "collecting"
                amount = min(b.level, truck.capacity - truck.load)
                collected = b.empty() if amount >= b.level else amount
                truck.load += collected
                self.stats["collected_total"] += collected
                truck.serviced_bins += 1
                truck.target_bin_id = None
                truck.state = "idle"
                return

        # Find a bin above threshold, choose nearest by path length
        candidates = [b for b in self.bins.values() if b.is_over_threshold(threshold)]
        if not candidates:
            # Idle if nothing to do
            truck.state = "idle"
            return

        # Greedy: choose path-shortest
        best = None
        best_path = None
        best_len = 1e18
        for b in candidates:
            path = astar_road_path(self.road, truck.pos, b.pos)
            if path is not None and len(path) < best_len:
                best = b
                best_path = path
                best_len = len(path)
        if best is not None and best_path is not None:
            truck.route = best_path
            truck.target_bin_id = best.id
            truck.state = "to_bin"
            return

        # If can't find path to any candidate, try dump
        self.route_to_nearest_dump(truck)
        truck.state = "to_dump"

    def route_to_nearest_dump(self, truck: Truck):
        best_path = None
        best_len = 1e18
        for d in self.dumps:
            path = astar_road_path(self.road, truck.pos, d)
            if path is not None and len(path) < best_len:
                best_len = len(path)
                best_path = path
        if best_path:
            truck.route = best_path

    def step_trucks(self):
        # First: planning
        for truck in self.trucks.values():
            self.plan_truck(truck)

        # Second: movement
        for truck in self.trucks.values():
            before_waits = truck.waits_at_red
            truck.step_move(self.road, self.traffic_light_at)
            self.stats["truck_waits"] += (truck.waits_at_red - before_waits)

    def step(self, steps=1):
        for _ in range(steps):
            self.t += 1
            self.step_traffic_lights()
            self.step_bins()
            self.step_trucks()



## Scenario Builders
Utilities to generate a mesh of roads, populate intersections with lights, place bins and dumps, and spawn trucks.


In [ ]:

def build_mesh_roads(sim: CitySim, spacing=6):
    sim.set_roads_mesh(spacing=spacing)
    # Place lights on intersections (where both row and col are multiples of spacing)
    for r in range(0, sim.h, spacing):
        for c in range(0, sim.w, spacing):
            # Skip edges to reduce light density
            if 0 < r < sim.h-1 and 0 < c < sim.w-1:
                sim.add_light(r, c, ns_green=10, ew_green=10, amber=2, initial=random.choice(["NS", "EW"]))

def place_random_bins(sim: CitySim, n_bins=60, min_dist_from_light=2, fill_rate_range=(0.2, 1.0)):
    placed = 0
    road_positions = np.argwhere(sim.road)
    lights_set = set(sim.lights.keys())
    while placed < n_bins and len(road_positions) > 0:
        r, c = tuple(road_positions[random.randrange(len(road_positions))])
        # Keep away from exact light cells
        if any(manhattan((r,c), L) < min_dist_from_light for L in lights_set):
            continue
        sim.add_bin((r,c), capacity=100.0, level=random.uniform(0, 50),
                    fill_rate=random.uniform(*fill_rate_range))
        placed += 1

def place_dumps(sim: CitySim, positions: Optional[List[Tuple[int,int]]] = None):
    if positions is None:
        # choose corners or near-corners that are on roads
        candidates = [(1,1), (1, sim.w-2), (sim.h-2, 1), (sim.h-2, sim.w-2)]
        for (r, c) in candidates:
            if sim.is_road((r,c)):
                sim.add_dump((r,c))
    else:
        for p in positions:
            if sim.is_road(p):
                sim.add_dump(p)

def spawn_trucks(sim: CitySim, n_trucks=4, capacity=500.0, speed=1):
    # Spawn trucks at dump positions (round-robin)
    if not sim.dumps:
        raise ValueError("No dumps present to spawn trucks.")
    for i in range(n_trucks):
        base = sim.dumps[i % len(sim.dumps)]
        sim.add_truck(base, capacity=capacity, speed=speed)



## Visualization
- Roads shown as a light gray backdrop  
- Bins as circles scaled by fill level (color = fill ratio)  
- Trucks as triangles  
- Dumps as squares  
- Traffic lights: small markers (green = permitted orientation, amber = pause)


In [ ]:

def draw_sim(sim: CitySim, ax=None, title: Optional[str]=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 7))
    ax.clear()

    # Background grid: roads
    bg = np.zeros((sim.h, sim.w))
    bg[sim.road] = 0.8  # light gray roads
    ax.imshow(bg, origin='lower', interpolation='nearest')

    # Bins: color by fill ratio, size by level
    if sim.bins:
        positions = np.array([b.pos for b in sim.bins.values()])
        levels = np.array([b.level for b in sim.bins.values()])
        ratios = np.array([b.level / b.capacity for b in sim.bins.values()])
        sizes = 30 + 120 * ratios  # scale bubble size
        sc = ax.scatter(positions[:,1], positions[:,0], s=sizes, c=ratios, cmap='viridis', marker='o', edgecolors='k', linewidths=0.3, alpha=0.9)
        cbar = plt.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label('Bin Fill Ratio')

    # Dumps
    if sim.dumps:
        dps = np.array(sim.dumps)
        ax.scatter(dps[:,1], dps[:,0], s=160, marker='s', edgecolors='k', facecolors='none', linewidths=1.0, label='Dump')

    # Traffic lights
    if sim.lights:
        for pos, tl in sim.lights.items():
            r, c = pos
            if tl.phase == "AMBER":
                ax.scatter(c, r, s=36, marker='X', color='tab:orange')
            elif tl.phase == "NS":
                ax.scatter(c, r, s=36, marker='|', color='tab:green')
            else:  # EW
                ax.scatter(c, r, s=36, marker='_', color='tab:green')

    # Trucks
    if sim.trucks:
        tpos = np.array([tr.pos for tr in sim.trucks.values()])
        ax.scatter(tpos[:,1], tpos[:,0], s=140, marker='^', color='tab:red', edgecolors='k', linewidths=0.6, label='Truck')

    # Cosmetics
    ax.set_xlim(-0.5, sim.w - 0.5)
    ax.set_ylim(-0.5, sim.h - 0.5)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_aspect('equal', 'box')
    if title is None:
        title = f"t={sim.t} | collected={sim.stats['collected_total']:.1f} | waits={sim.stats['truck_waits']}"
    ax.set_title(title)

    # Legend slim
    ax.legend(loc='upper right', fontsize=8, frameon=True)
    return ax

def animate_sim(sim: CitySim, steps=200, render_every=1):
    fig, ax = plt.subplots(figsize=(10,7))
    for k in range(steps):
        sim.step(1)
        if k % render_every == 0:
            draw_sim(sim, ax=ax)
            plt.pause(0.001)
    plt.show()



## Interactive Panel (Optional)
If `ipywidgets` is available, use the panel below to **step** or **run** the simulation interactively.


In [ ]:

def interactive_panel(sim: CitySim):
    if not HAS_WIDGETS:
        print("ipywidgets not available. Falling back to non-interactive usage.")
        return

    step_slider = IntSlider(description='Steps/Click', min=1, max=20, value=5)
    run_steps = IntSlider(description='Run Steps', min=10, max=500, value=100)
    btn_step = Button(description='Step')
    btn_run = Button(description='Run')
    btn_redraw = Button(description='Redraw')
    out = None

    fig, ax = plt.subplots(figsize=(10,7))
    draw_sim(sim, ax=ax)

    def on_step(_):
        sim.step(step_slider.value)
        draw_sim(sim, ax=ax)
        plt.show()

    def on_run(_):
        for _ in range(run_steps.value):
            sim.step(1)
        draw_sim(sim, ax=ax)
        plt.show()

    def on_redraw(_):
        draw_sim(sim, ax=ax)
        plt.show()

    btn_step.on_click(on_step)
    btn_run.on_click(on_run)
    btn_redraw.on_click(on_redraw)

    display(VBox([HBox([step_slider, btn_step, run_steps, btn_run, btn_redraw])]))



## Scenario Setup
Create a city with a mesh of roads, intersections with lights, random bins, dumps at corners, and a fleet of trucks.


In [ ]:

# --- Create a default scenario ---
sim = CitySim(height=40, width=60)
build_mesh_roads(sim, spacing=6)
place_dumps(sim)                    # default to corners on roads
place_random_bins(sim, n_bins=80, min_dist_from_light=2, fill_rate_range=(0.3, 1.2))
spawn_trucks(sim, n_trucks=6, capacity=600.0, speed=1)

print(f"Roads: {sim.road.sum()} cells | Lights: {len(sim.lights)} | Bins: {len(sim.bins)} | Dumps: {len(sim.dumps)} | Trucks: {len(sim.trucks)}")



## Run & Visualize
Use either the **simple loop** (fast) or the **interactive panel** (if available).


In [ ]:

# Quick run preview (static updates in-place)
# for _ in range(50):
#     sim.step(1)
# draw_sim(sim)

# Or animate live (blocking):
# animate_sim(sim, steps=300, render_every=2)

# Or try the interactive panel:
# interactive_panel(sim)



## Policy API (Extend Me)
You can override the default greedy assignment with a smarter multi‑agent policy.
Below is an example hook that **pre-assigns** bins to trucks (simple round‑robin among top‑K fullest bins).
Call `use_round_robin_policy(sim)` to enable.


In [ ]:

def use_round_robin_policy(sim: CitySim, top_k: int = 20, threshold: float = 0.5):
    """Replace CitySim.plan_truck with a policy that pre-assigns bins in round-robin among trucks.
    This is a minimal example to illustrate how to plug policies.
    """
    # Snapshot fullest bins above threshold
    cand = [b for b in sim.bins.values() if b.is_over_threshold(threshold)]
    cand.sort(key=lambda b: b.level / b.capacity, reverse=True)
    cand = cand[:top_k]

    # Build a mutable queue of bin ids
    q = deque([b.id for b in cand])

    original_plan_truck = sim.plan_truck

    def policy_plan_truck(truck: Truck, threshold=threshold):
        # If near full, dump
        if truck.load >= truck.capacity * 0.9:
            sim.route_to_nearest_dump(truck)
            truck.state = "to_dump"
            return

        # If en-route, keep going
        if truck.route and not truck.at_destination():
            return

        # Unload if at dump
        if truck.pos in sim.dumps and truck.load > 0 and truck.at_destination():
            truck.state = "unloading"
            truck.load = 0.0
            truck.target_bin_id = None
            truck.state = "idle"
            return

        # Collect if arrived at target bin
        if truck.target_bin_id is not None and truck.at_destination():
            b = sim.bins.get(truck.target_bin_id, None)
            if b and b.pos == truck.pos:
                truck.state = "collecting"
                amount = min(b.level, truck.capacity - truck.load)
                collected = b.empty() if amount >= b.level else amount
                truck.load += collected
                sim.stats["collected_total"] += collected
                truck.serviced_bins += 1
                truck.target_bin_id = None
                truck.state = "idle"
                return

        # Otherwise, try to pop a pre-assigned bin
        for _ in range(len(q)):
            bid = q[0]
            q.rotate(-1)
            b = sim.bins.get(bid, None)
            if b is None or not b.is_over_threshold(threshold):
                continue
            path = astar_road_path(sim.road, truck.pos, b.pos)
            if path:
                truck.route = path
                truck.target_bin_id = b.id
                truck.state = "to_bin"
                return

        # Fallback to original plan
        original_plan_truck(truck, threshold=threshold)

    sim.plan_truck = policy_plan_truck



## Save / Load (Basic)
Minimal snapshot utilities (roads and agent states). Extend as needed.


In [ ]:

import json

def save_state(sim: CitySim, path: str):
    data = {
        "h": sim.h, "w": sim.w,
        "road": sim.road.astype(int).tolist(),
        "t": sim.t,
        "lights": [
            {"pos": list(tl.pos), "ns": tl.ns_green_steps, "ew": tl.ew_green_steps, "amber": tl.amber_steps, "phase": tl.phase, "timer": tl.timer}
            for tl in sim.lights.values()
        ],
        "bins": [
            {"id": b.id, "pos": list(b.pos), "capacity": b.capacity, "level": b.level, "fill_rate": b.fill_rate}
            for b in sim.bins.values()
        ],
        "dumps": [list(d) for d in sim.dumps],
        "trucks": [
            {"id": tr.id, "pos": list(tr.pos), "capacity": tr.capacity, "load": tr.load, "speed": tr.speed_cells_per_step}
            for tr in sim.trucks.values()
        ],
        "stats": sim.stats,
    }
    with open(path, "w") as f:
        json.dump(data, f)

def load_state(path: str) -> CitySim:
    with open(path, "r") as f:
        data = json.load(f)
    sim = CitySim(data["h"], data["w"])
    sim.road = np.array(data["road"], dtype=bool)
    for L in data["lights"]:
        pos = tuple(L["pos"])
        sim.add_light(pos[0], pos[1], ns_green=L["ns"], ew_green=L["ew"], amber=L["amber"], initial=L["phase"])
        sim.lights[pos].timer = L["timer"]
    for B in data["bins"]:
        sim.bins[B["id"]] = Bin(B["id"], tuple(B["pos"]), B["capacity"], B["level"], B["fill_rate"])
    sim.dumps = [tuple(d) for d in data["dumps"]]
    for T in data["trucks"]:
        tr = Truck(T["id"], tuple(T["pos"]), T["capacity"], T["load"], T["speed"])
        sim.trucks[tr.id] = tr
    sim.stats = data.get("stats", {"collected_total": 0.0, "truck_waits": 0})
    return sim



## Notes & Next Steps
- **Coordination**: Replace the greedy policy with assignment (e.g., Hungarian) or auction‑based coordination.  
- **Road network**: Import from a real map or manually design complex networks.  
- **Traffic lights**: Add turn lanes, amber behavior, or probabilistic compliance.  
- **Travel time**: Use speed limits; make roads multi‑lane; penalize turns.  
- **Bin dynamics**: Make fill rates time‑of‑day dependent.  
- **Stochasticity**: Add incident events (road closures), truck breakdowns, or bin overflows.  
- **Metrics**: Track per‑truck utilization, on‑time service, unmet demand, and energy use.  
- **RL**: Wrap the environment as a Gymnasium interface for learning policies.
